**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Jul  2 06:17:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 b_depressed.csv
'Colab Notebooks'
 data_img
 data_target.csv
 face123_new_model.h5
 face124_new_model.h5
 face.jpg
 features.csv
 features_mask.csv
 fer2013_test_new1.csv
 fer2013_train_new1.csv
 geological_similarity
'Getting started.pdf'
 IMG_20180605_063830.jpg
 IMG_20180605_080407.jpg
 IMG_20180823_181538.JPG
 IMG_20180905_124956.jpg
 IMG_20180909_164653.jpg
 IMG_20180909_182006.jpg
 IMG_20180913_114809.jpg
 IMG_20180913_144912.jpg
 IMG_20180913_193312.j

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13781, done.
remote: Total 13781 (delta 0), reused 0 (delta 0), pack-reused 13781
Receiving objects: 100% (13781/13781), 12.37 MiB | 6.96 MiB/s, done.
Resolving deltas: 100% (9405/9405), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "plate" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-02 06:20:45--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   171KB/s    in 21m 49s 

2020-07-02 06:42:36 (121 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/_7043ee18-cae9-11e9-b728-1326979780a6.jpg  
  inflating: data/obj/_7043ee18-cae9-11e9-b728-1326979780a6.txt  
  inflating: data/obj/_dsc0438.jpg   
  inflating: data/obj/_dsc0438.txt   
  inflating: data/obj/0.jpg          
  inflating: data/obj/0.txt          
  inflating: data/obj/1481638504lwTVweOdb4oWfIOm0dNM.jpg  
  inflating: data/obj/1481638504lwTVweOdb4oWfIOm0dNM.txt  
  inflating: data/obj/1513f8d4a01ded0af61ebed80fbb3bf2.jpg  
  inflating: data/obj/1513f8d4a01ded0af61ebed80fbb3bf2.txt  
  inflating: data/obj/1557294285-3578 (1).jpg  
  inflating: data/obj/1557294285-3578 (1).txt  
  inflating: data/obj/1557294285-3578.jpg  
  inflating: data/obj/1557294285-3578.txt  
  inflating: data/obj/163594133_15644787681_large.jpg  
  inflating: data/obj/163594133_15644787681_large.txt  
  inflating: data/obj/18seater.jpg   
  inflating: data/obj/18seater.txt   
  inflating: data/obj/2011_Skoda_Superb_Review (1).jpg  
  inflatin

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.720000 0.562232 0.205714 0.077253

['15', '0.720000', '0.562232', '0.205714', '0.077253']
0 data/obj/1557294285-3578 (1).txt
0 0.720000 0.562232 0.205714 0.077253
15 0.210000 0.735000 0.160000 0.096667

['15', '0.210000', '0.735000', '0.160000', '0.096667']
1 data/obj/my-mahindra-cj640-21230921.txt
0 0.210000 0.735000 0.160000 0.096667
15 0.222408 0.656805 0.190635 0.118343

['15', '0.222408', '0.656805', '0.190635', '0.118343']
2 data/obj/images (4).txt
0 0.222408 0.656805 0.190635 0.118343
15 0.505792 0.530928 0.849421 0.247423

['15', '0.505792', '0.530928', '0.849421', '0.247423']
3 data/obj/images (5).txt
0 0.505792 0.530928 0.849421 0.247423
15 0.683000 0.855956 0.074000 0.077562

['15', '0.683000', '0.855956', '0.074000', '0.077562']
4 data/obj/bus_6.txt
0 0.683000 0.855956 0.074000 0.077562
15 0.322000 0.850416 0.084000 0.072022

['15', '0.322000', '0.850416', '0.084000', '0.072022']
4 data/obj/bus_6.txt
0 0.322000 0.850416 0.084000 0.072022
15 0.718750 0.807528 0.128906 0

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/2012-skoda-octavia-exclusive-exterior-front-left.jpg', 'data/obj/images (11).jpg', 'data/obj/jay-bhavani-tours-and-travels-yerawada-pune-tempo-travellers-on-hire-f3950.jpg', 'data/obj/2011-010-09-car-2.jpg', 'data/obj/Car.jpg', 'data/obj/download.jpg', 'data/obj/_dsc0438.jpg', 'data/obj/India_Trip_2009.jpg', 'data/obj/0.jpg', 'data/obj/images (6).jpg', 'data/obj/New-2017-Skoda-Octavia-Review-Fascia-1.jpg', 'data/obj/E-Class-vs-5-Series-vs-A6-vs-XF.jpg', 'data/obj/decorated-indian-tata-truck-on-a-dirt-road-in-maharashtra-india-CXXB41.jpg', 'data/obj/maxresdefault.jpg', 'data/obj/CTXVpRBWIAAKNBK.jpg', 'data/obj/23809370439_58546d8103_b.jpg', 'data/obj/jai-bhavani-tours-and-travels-thergaon-pune-tempo-travellers-on-hire-for-outstation-4l1vayx.jpg', 'data/obj/humorous-car-number-plates-in-west-london-britain-shutterstock-editorial-955018a.jpg', 'data/obj/main-qimg-d3f8003c7b707bd633b15f6d31c79728-c.jpg', 'data/obj/maruti-super-carry-close-up-shot.jpg', 'data/obj/ashok-leyland-tr

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 49933, rewritten_bbox = 0.656880 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.491812, GIOU: 0.412660), Class: 0.992408, Obj: 0.407759, No Obj: 0.000313, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.087759, iou_loss = 0.146712, total_loss = 0.234471 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.694043, GIOU: 0.666021), Class: 0.998796, Obj: 0.838275, No Obj: 0.000347, .5R: 1.000000, .75R: 0.000000, count: 2, class_loss = 0.094225, iou_loss = 0.133510, total_loss = 0.227735 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.490906, GIOU: 0.431068), Class: 0.971368, Obj: 0.425445, No Obj: 0.000078, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.185098, iou_loss = 0.101333, total_loss = 0.286432 
 total_bbox = 49937, rewritten_bbox = 0.656828 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0